### Location Preprocessor

In [1]:
def get_provinces_list():
   with open('../res/raw/provinces.txt') as province_file:
      return [province
            .lower()
            .replace('\n','') 
            .strip()
            for province in province_file.readlines()]

In [2]:
def get_tourist_dest_list():
   with open('../res/raw/tourist_dest.txt') as tourist_dest_file:
      return [tourist_dest
              .replace('\n', '')
              for tourist_dest in tourist_dest_file.readlines()]


In [3]:
import pandas as pd
def get_cities_list():
   raw_cities_df = pd.read_csv('../res/raw/worldcities.csv', index_col=False)
   ph_cities_df = raw_cities_df.loc[raw_cities_df['iso3'] == 'PHL']
   ph_cities_df = ph_cities_df[['city']]
   ph_cities_df = ph_cities_df['city']
   return ph_cities_df.to_list()

In [4]:
print('READING PROVINCE LIST')
provinces_list = get_provinces_list()
print('READING TOURIST DESTINATION LIST')
cities_list = get_cities_list()
print('READING CITIES LIST')
tourist_dest_list = get_tourist_dest_list()

print('COMPLETED READING FILES')

all_locations = provinces_list+cities_list+tourist_dest_list 
all_locations = [loc.lower() for loc in all_locations]

# Check if a location exists
'boracay' in ''.join(all_locations)

READING PROVINCE LIST
READING TOURIST DESTINATION LIST
READING CITIES LIST
COMPLETED READING FILES


True

### Peoples Names preprocessor

['dela cruz',
 'garcia',
 'reyes',
 'ramos',
 'mendoza',
 'santos',
 'flores',
 'gonzales',
 'bautista',
 'villanueva',
 'fernandez',
 'cruz',
 'de guzman',
 'lopez',
 'perez',
 'castillo',
 'francisco',
 'rivera',
 'aquino',
 'castro',
 'sanchez',
 'torres',
 'de leon',
 'domingo',
 'martinez',
 'rodriguez',
 'santiago',
 'soriano',
 'delos santos',
 'diaz',
 'hernandez',
 'tolentino',
 'valdez',
 'ramirez',
 'morales',
 'mercado',
 'tan',
 'aguilar',
 'navarro',
 'manalo',
 'gomez',
 'dizon',
 'del rosario',
 'javier',
 'corpuz',
 'gutierrez',
 'salvador',
 'velasco',
 'miranda',
 'david',
 'salazar',
 'ferrer',
 'alvarez',
 'sarmiento',
 'pascual',
 'lim',
 'delos reyes',
 'marquez',
 'jimenez',
 'cortez',
 'antonio',
 'agustin',
 'rosales',
 'manuel',
 'mariano',
 'evangelista',
 'pineda',
 'enriquez',
 'ocampo',
 'alcantara',
 'pascua',
 'de vera',
 'romero',
 'de jesus',
 'dela peña',
 'valencia',
 'ignacio',
 'vergara',
 'padilla',
 'angeles',
 'espiritu',
 'fuentes',
 'legaspi'